# MA5701 Optimización no lineal: Tarea 3
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [20]:
from scipy.optimize import linprog
import numpy as np
!pip install numdifftools
import numdifftools as nd
import re

#  Método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

## Paso 0

In [213]:
def step0(xk, A, b, E=None, e=None):
    # Step 0: Check factibilidad
    feasible = np.all(A@xk <= b)
    if str(E) != "None":
        feasible = feasible and np.all(np.isclose(E@xk, e))
    return feasible

## Paso 1

In [5]:
def step1(xk, A, b):
    # Step 1
    index_A1 = np.isclose(A@xk, b)
    index_A2 = ~index_A1

    A1 = A[index_A1]
    A2 = A[index_A2]

    b1 = b[index_A1]
    b2 = b[index_A2]
    
    return A1, A2, b1, b2

## Paso 2

In [6]:
def step2(obj, A1, E, eps):
    # Step 2
    lhs_ineq = A1
    rhs_ineq = np.zeros(A1.shape[0]) if str(A1) != "None" else None

    lhs_eq = E
    rhs_eq = np.zeros(E.shape[0]) if str(E) != "None" else None

    bnd = [(-1, 1) for _ in range(obj.shape[0])] 

    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                  A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
                  method="simplex")
    dk = opt.x
    stop = abs(np.dot(obj, dk))<eps
    return dk, stop

## Paso 3

In [179]:
def step3(f, xk, dk, A2, b2, eps, N=10*6, sgm=0.5):
    # Step 3
    # lambda maximo
    dom = A2@dk>0
    bar_lmbdk = np.min(((b2-A2@xk)/(A2@dk))[dom]) if len(dom) != 0 else float("inf")

    # paso de armijo
    t = np.linspace(0, bar_lmbdk, N)
    h = 1/(N-1)

    d_phi_0 = nd.Gradient(lambda t: f( xk + dk*t ))([0])

    dom = np.where(f( (xk + dk * t[:, None]).T ) <= f(xk) + sgm*d_phi_0*t)[0]
    ix_max = np.max(dom)
    lmbdk = ix_max * h * bar_lmbdk

    # nuevo paso
    xk1 = xk + lmbdk * dk
    stop = lmbdk<eps
    return xk1, stop

In [244]:
class Zoutendijk(object):
    
    def __init__(self, fun: str, const: list):
        self.fun = fun
        self.const = const
        
        self.set_vars()
        self.set_f()
        self.set_A_b_E_e()
    
    def set_vars(self):
        self.vars = set(re.findall("x\d", self.fun))
        for c in self.const:
            self.vars = self.vars.union(set(re.findall("x\d", c)))   
        self.vars = sorted(list(self.vars))
        self.dic_vars = {var: f"x[{k}]" for k, var in enumerate(self.vars)}
    
    def set_f(self):
        self.rename_fun = self.fun
        for k, v in self.dic_vars.items():
            self.rename_fun = self.rename_fun.replace(k, v)
        
    def f(self, x):
        return eval(self.rename_fun)
    
    def set_A_b_E_e(self):
        n = len(self.dic_vars)
        m = len(self.const)
        m_A = sum(int("<" in c or ">" in c) for c in self.const)
        m_E = m-m_A

        self.A = None
        self.E = None

        self.b = None
        self.e = None

        if m_A:
            self.A = np.zeros((m_A, n))
            self.b = np.zeros(m_A)
        if m_E:
            self.E = np.zeros((m_E, n))
            self.e = np.zeros(m_E)

        i_A = 0
        i_E = 0
        for c in self.const:
            if "<" in c or ">" in c:
                sgn = 1 if "<" in c else -1
                left, right = c.split("<=") if sgn>0 else c.split(">=")
                self.b[i_A] = sgn*float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.A[i_A, j] = sgn
                        elif t_2 == t_1 == "" and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                        elif t_2 in ["", "+"] and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.A[i_A, j] = sgn
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_A+=1
            else:
                left, right = c.split("=")
                self.e[i_E] = float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.E[i_E, j] = 1
                        elif t_2 == t_1 == "" and t == "-":
                            self.E[i_E, j] = -1
                        elif t_2 in ["", "+"] and t == "*":
                            self.E[i_E, j] = float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.E[i_E, j] = float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.E[i_E, j] = 1
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.E[i_E, j] = -1
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_E+=1
                
    def print_status(self, feasible, xk, f_xk, step=0, k=0):
        print(f"(step={step}, k={k}) Factible: {feasible}")
        print(f"(step={step}, k={k}) xk: {xk}")
        print(f"(step={step}, k={k}) f(xk): {f_xk}\n")
                
    def solve(self, x0, eps=1e-6, N=10*6, sgm=0.5, verbose=True):
        # Step 0: Inicializar
        k = 0
        xk = x0
        grad = nd.Gradient(self.f)
        ## Step 0: Check factibilidad
        feasible = step0(xk, self.A, self.b, self.E, self.e)
        stop = not feasible
        ## Step 0: Imprimir estado
        f_xk = self.f(xk)
        if verbose:
            self.print_status(feasible, xk, f_xk, step=0, k=k)
        list_xk = [xk]
        list_f_xk = [f_xk]    
        while not stop:
            # Step 1: Descomposicion [A1, A2]
            A1, A2, b1, b2 = step1(xk, self.A, self.b)
            # Step 2: Resolver problema lineal (Dk)
            obj = grad(xk)
            dk, stop_grad = step2(obj, A1, self.E, eps)
            stop = stop_grad
            if not stop:
                # Step 3: Resolver problema (L)
                xk1, stop_lambda = step3(self.f, xk, dk, A2, b2, eps, N=N, sgm=sgm)
                xk = xk1
                feasible = step0(xk, self.A, self.b, self.E, self.e)
                stop = (not feasible) or stop_lambda
                ## Step 3: Imprimir estado
                f_xk = self.f(xk)
                if verbose:
                    self.print_status(feasible, xk, f_xk, step=3, k=k)
                k+=1
                list_xk.append(xk)
                list_f_xk.append(f_xk)

In [245]:
method1 = Zoutendijk(
    fun="8 * ( x1 - 6 ) ** 2 + ( x2 - 2 ) ** 4", 
    const=[
        "- x1 + 2 * x2 <= 4",
        "3 * x1 + 2 * x2 <= 12",
        "x1 >= 0",
        "x2 >= 0"
    ]
)

method1.solve(
    x0=[0, 2], 
    eps=1e-6, 
    N=10*9, 
    sgm=0.5, 
    verbose=True
)

(step=0, k=0) Factible: True
(step=0, k=0) xk: [0, 2]
(step=0, k=0) f(xk): 288

(step=3, k=0) Factible: True
(step=3, k=0) xk: [2. 3.]
(step=3, k=0) f(xk): 129.0

(step=3, k=1) Factible: True
(step=3, k=1) xk: [4. 0.]
(step=3, k=1) f(xk): 48.000000000000014

(step=3, k=2) Factible: True
(step=3, k=2) xk: [3.86516854 0.20224719]
(step=3, k=2) f(xk): 46.90531853970598

(step=3, k=3) Factible: True
(step=3, k=3) xk: [3.86516854 0.20224719]
(step=3, k=3) f(xk): 46.90531853970598



In [246]:
method2 = Zoutendijk(
    fun="x1 ** 4 - 2 * x2 ** 2 + 10 * x1 * ( x2 ** 2 ) + x4 ** 2", 
    const=[
        "x1 + x2 - x3 = 1",
        "x1 + x4 = 4",
        "x1 - x2 = 0",
        "x1 >= 0",
        "x2 >= 0",
        "x3 >= 0",
        "x4 >= 0"
    ]
)
method2.solve(
    x0=[2, 2, 3, 2], 
    eps=1e-6, 
    N=10*9, 
    sgm=0.5, 
    verbose=True
)

(step=0, k=0) Factible: True
(step=0, k=0) xk: [2, 2, 3, 2]
(step=0, k=0) f(xk): 92

(step=3, k=0) Factible: True
(step=3, k=0) xk: [0.92134831 0.92134831 0.84269663 3.07865169]
(step=3, k=0) f(xk): 16.322099237736822

(step=3, k=1) Factible: True
(step=3, k=1) xk: [0.57574801 0.57574801 0.15149602 3.42425199]
(step=3, k=1) f(xk): 13.080935697957145

(step=3, k=2) Factible: True
(step=3, k=2) xk: [0.53234185 0.53234185 0.0646837  3.46765815]
(step=3, k=2) f(xk): 13.046778123625602

(step=3, k=3) Factible: True
(step=3, k=3) xk: [0.53125167 0.53125167 0.06250335 3.46874833]
(step=3, k=3) f(xk): 13.046753890147809

(step=3, k=4) Factible: True
(step=3, k=4) xk: [0.53125167 0.53125167 0.06250335 3.46874833]
(step=3, k=4) f(xk): 13.046753890147809

